<a href="https://colab.research.google.com/github/peterruler/apertus/blob/main/apertus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# Step 1: Set CUDA_HOME environment variable (if not already set or correct)
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'

# Step 2: Update pip and setuptools
!pip install --upgrade pip setuptools

# Step 3: Install build essentials (if not already present in Colab's base image)
# Colab usually has these, but it doesn't hurt to ensure.
!sudo apt-get update
!sudo apt-get install -y build-essential

# Step 4: Try installing xielu with --no-build-isolation
# Replace 'xielu' with the actual installation command you are using
# e.g., if you are cloning a repo and installing from there:
# !git clone <xielu_repo_url>
# %cd <xielu_repo_directory>
# !pip install . --no-build-isolation --no-deps

# If you are installing directly by name:
!pip install git+https://github.com/nickjbrowning/XIELU --no-build-isolation

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [4]:
!pip install -U bitsandbytes

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "swiss-ai/Apertus-8B-Instruct-2509"
device = "cuda"  # for GPU usage or "cpu" for CPU usage

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16, # Load the model in bfloat16
    load_in_8bit=True, # Load the model in 8-bit
)

# prepare the model input
prompt = "Du bist ein Philosoph und Astrophysiker. Erkläre einem 10 jährigen Kind in maximal 100 Wötern, wie Aristoteles die Astrophysik der Planetenumlaufbahnen erklären würde."
messages_think = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages_think,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt", add_special_tokens=False).to(model.device)

# Generate the output
generated_ids = model.generate(**model_inputs, max_new_tokens=32768)

# Get and decode the output
output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Liebe(r) kleine(r) Freund(in), stell dir vor, du spielst mit vielen, vielen Bällen. Jeder Ball steht für einen Planeten, der sich im Weltraum bewegt. Aristoteles, ein alter griechischer Philosoph, würde sagen, dass die Bälle (Planeten) sich auf bestimmte Bahnen bewegen, die von Anfang an festgelegt sind. Stell dir vor, die Bälle fliegen in perfekten Kreisen oder Ellipsen, ohne jemals aus diesen Bahnen auszubrechen. Das ist, wie Aristoteles die Planetenumlaufbahnen erklären würde.
